## Data prep.

In [41]:
# import packages
import pandas as pd
import numpy as np
from numpy import where
from sklearn.impute import KNNImputer
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split

In [42]:
# connect the drive
df = pd.read_excel('/content/drive/MyDrive/MMAI/Finance/Bankruptcy_data_Final.xlsx')
#df = pd.read_csv('Bankruptcy_data_final.csv')

In [43]:
# look at shape
print(df.shape)
df.head(3)

(92872, 13)


,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
0,1.58,0.36,0.18,0.13,1.33,1.77,0.07,0.15,2.22,NaN,NaN,NaN,0
1,1.41,0.36,0.19,0.12,1.31,1.59,0.07,0.13,2.41,0.126,0.014,0.040,0
2,0.31,0.32,0.13,0.08,1.03,1.55,0.05,0.04,2.56,0.368,0.328,0.567,0


In [44]:
print('Full dataset target distribution...')
print('1:', df['BK'][df['BK']==1].count(),',',df['BK'][df['BK']==1].count()/df['BK'].count() )
print('0:', df['BK'][df['BK']==0].count(),',',df['BK'][df['BK']==0].count()/df['BK'].count() )

Full dataset target distribution...
1: 558 , 0.00600826944611939
0: 92314 , 0.9939917305538806


In [45]:
# check out the columns
print('Understanding columns...')
for col in df.columns:
  print(col,':')
  print('Total nulls:',df[col].isnull().values.sum())
  print('Min:',df[col].min())
  print('Max:',df[col].max())
  if df[col].isnull().values.sum() == 0:
    print('All good for', col)
  elif df[col].isnull().values.sum() <= 30:
    print('Drop the nulls for', col,'\n')
  else:
    print('Use imputation for', col,'\n')

Understanding columns...
EPS :
Total nulls: 5
Min: -384000.0
Max: 55339.0
Drop the nulls for EPS 

Liquidity :
Total nulls: 247
Min: -25968.52
Max: 1.0
Use imputation for Liquidity 

Profitability :
Total nulls: 247
Min: -79682.0
Max: 140.58
Use imputation for Profitability 

Productivity :
Total nulls: 247
Min: -5093.0
Max: 1102.0
Use imputation for Productivity 

Leverage Ratio :
Total nulls: 26
Min: -7811.0
Max: 75970.38
Drop the nulls for Leverage Ratio 

Asset Turnover :
Total nulls: 247
Min: -31.59
Max: 276.38
Use imputation for Asset Turnover 

Operational Margin :
Total nulls: 5557
Min: -30175.7
Max: 394.47
Use imputation for Operational Margin 

Return on Equity :
Total nulls: 8
Min: -88875.14
Max: 39500.0
Drop the nulls for Return on Equity 

Market Book Ratio :
Total nulls: 57
Min: -3151500.0
Max: 3455419.33
Use imputation for Market Book Ratio 

Assets Growth :
Total nulls: 6701
Min: -1.0
Max: 14231.0
Use imputation for Assets Growth 

Sales Growth :
Total nulls: 6701
Min: 

In [46]:
# drop nulls where only a few missing on entire dataset
# same thing with outliers
print('All:',df.shape)
df = df.dropna(subset=['Return on Equity','Leverage Ratio','EPS'])
print('Nulls dropped where very few:',df.shape)

# Remove outliers that are not 1 target anyways
df_a = df[(df['Employee Growth']<=1000) | (df['Employee Growth'].isnull())] 
df_a = df_a[(df_a['Sales Growth']<=1000) | (df_a['Sales Growth'].isnull())] 
df_a = df_a[(df_a['Sales Growth']>=-10) | (df_a['Sales Growth'].isnull())] 
df_a = df_a[(df_a['Return on Equity']<=10000) | (df_a['Return on Equity'].isnull())] 
df_a = df_a[(df_a['Return on Equity']>=-25000) | (df_a['Return on Equity'].isnull())] 
df_a = df_a[(df_a['Operational Margin']<=11) | (df_a['Operational Margin'].isnull())] 
df_a = df_a[(df_a['Asset Turnover']>=-2) | (df_a['Asset Turnover'].isnull())]  
df_a = df_a[(df_a['Leverage Ratio']<=3500) | (df_a['Leverage Ratio'].isnull())]
df_a = df_a[(df_a['Leverage Ratio']>=-3500) | (df_a['Leverage Ratio'].isnull())]
df_a = df_a[(df_a['Productivity']<=50) |  (df_a['Productivity'].isnull())]
df_a = df_a[(df_a['Profitability']<=6) |  (df_a['Profitability'].isnull())]  
df_a = df_a[(df_a['Liquidity']>=-15000)  |  (df_a['Liquidity'].isnull())]  
print('Outliers dropped, none were target or null = 1:',df_a.shape)

All: (92872, 13)
Nulls dropped where very few: (92839, 13)
Outliers dropped, none were target or null = 1: (92805, 13)


# Split Train/Test

In [67]:
# split training data
y = df['BK']
X = df.drop(columns=['BK'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15,
                                                    random_state=42)
print('Training:',X_train.shape)
print('Testing:',X_test.shape)

Training: (78913, 12)
Testing: (13926, 12)


# Scale/Fill Missing Values/Sampling

# Scale

In [68]:
#mm_scaler = MinMaxScaler()
scaler = PowerTransformer(method='yeo-johnson')
X_train_scaled = scaler.fit_transform(X_train)

# Fill Missing Values

In [69]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10,random_state=42)
imp.fit(X_train_scaled)
X_train_imp = pd.DataFrame(imp.transform(X_train_scaled),columns=X_train.columns)

In [70]:
# look at the imputed version
X_train_imp.head(3)

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth
0,-0.001093,-0.880589,0.029763,0.033980,0.000414,-1.003668,0.064453,0.002951,-0.013589,-0.065850,-0.028484,-0.066348
1,0.000297,-0.092123,0.160495,0.036459,0.000035,0.731348,0.061243,0.003369,-0.016493,0.327063,0.024412,0.498914
2,-0.000723,-0.035140,0.111262,0.035838,0.005629,1.127889,0.060608,0.003189,-0.016380,-0.039046,0.024061,0.004879


In [71]:
# see shapes
counter = Counter(y_train)
print('Training distribution:',counter)
print('1:', y_train[y_train==1].count()/y_train.count() )
print('0:', y_train[y_train==0].count()/y_train.count() )

ycounter = Counter(y_test)
print('Testing distribution:',ycounter)
print('1:', y_test[y_test==1].count()/y_test.count() )
print('0:', y_test[y_test==0].count()/y_test.count() )

Training distribution: Counter({0: 78438, 1: 475})
1: 0.006019287062968079
0: 0.9939807129370319
Testing distribution: Counter({0: 13847, 1: 79})
1: 0.005672842165733161
0: 0.9943271578342668


# Oversampling

In [72]:
# transform the training dataset with smote /and under sampling
oversample = SMOTE(sampling_strategy=0.05) #.1
X_smote, y_smote = oversample.fit_resample(X_train_imp, y_train)
counter = Counter(y_smote)
print('New distribution:',counter)


New distribution: Counter({0: 78438, 1: 3921})


In [73]:
# New features
def pos(x):
  if x > 0:
    y = 1
  else: 
    y = 0
  return y

def beyond_one(x):
  if x >= 1:
    y = 1
  else: 
    y = 0
  return y

## Modeling

In [74]:
# import packages
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
# from sklearn.feature_selection import RFE

In [75]:
# try a model to make sure cleaned ok for next team member
#stnd_scaler = StandardScaler()

Pipe= Pipeline([('rf', RandomForestClassifier())])
param_grid = {'rf__criterion': ['entropy']
              }
model = GridSearchCV(Pipe,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
model.fit(X_smote,y_smote) #X_smote, y_smote
print('\n\nBest Score:', model.best_score_,'\n')
print('Best parameters:', model.best_params_)



Best Score: 0.993357616294308 

Best parameters: {'rf__criterion': 'entropy'}


In [86]:
from sklearn.metrics import roc_auc_score
'''
nan = np.nan
imputer = KNNImputer(n_neighbors=4, weights="distance") 
v_array_imputed = imputer.fit_transform(X_test)
X_test_imp = pd.DataFrame(v_array_imputed, columns = X_test.columns)
print('Any nulls left?:', X_test_imp.isnull().values.sum(),'\n')
print('Testing shape:',X_test_imp.shape,'\n')
'''

#impute with iterative imputer and add features
X_test_scaled = scaler.transform(X_test)
X_test_imp = imp.transform(X_test_scaled)
X_test_imp  = pd.DataFrame(X_test_imp , columns = X_test.columns )
#X_test_imp['Sales_More_Assets'] = (X_test_imp['Sales Growth'] - X_test_imp['Assets Growth']).apply(pos)
#X_test_imp['Pos_Liquidity'] = X_test_imp['Liquidity'].apply(pos)
#X_test_imp['Pos_Profitability'] = X_test_imp['Profitability'].apply(pos)
#X_test_imp['High_Leverage'] = X_test_imp['Leverage Ratio'].apply(beyond_one)

#predict
pred = model.predict_proba(X_test_imp)[:,1]
threshold = 0.009
print('Confusion matrix:\n',confusion_matrix(y_test, pred>threshold))
print('\nClassification report:\n',classification_report(y_test, pred>threshold))
print (f'AUC Score: {roc_auc_score(y_test,pred>threshold)}')
# prooving credit upfront -> recall
# else precision

Confusion matrix:
 [[9698 4149]
 [   3   76]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.70      0.82     13847
           1       0.02      0.96      0.04        79

    accuracy                           0.70     13926
   macro avg       0.51      0.83      0.43     13926
weighted avg       0.99      0.70      0.82     13926

AUC Score: 0.8311968136405727
